In [1]:
import pandas as pd
import os
import cv2

In [2]:
full_path_to_ts_dataset = os.path.join(os.getcwd()+'\..\Images\Detection\PPM format\\')
full_path_to_ts_dataset_info = os.path.join(os.getcwd()+'\..\Images\Detection\\')
# print(full_path_to_ts_dataset)

In [3]:
# Defining lists for categories according to the classes ID's
# Prohibitory category:
# circular Traffic Signs with white background and red border line
p = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 15, 16]

# Danger category:
# triangular Traffic Signs with white background and red border line
d = [11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

# Mandatory category:
# circular Traffic Signs with blue background
m = [33, 34, 35, 36, 37, 38, 39, 40]

# Other category:
o = [6, 12, 13, 14, 17, 32, 41, 42]

In [4]:
ann = pd.read_csv(full_path_to_ts_dataset_info + '/' + 'DataInfo.txt',
                  names=['ImageID',
                         'XMin',
                         'YMin',
                         'XMax',
                         'YMax',
                         'ClassID'],
                  sep=';')
print(ann.head())

     ImageID  XMin  YMin  XMax  YMax  ClassID
0  00000.ppm   774   411   815   446       11
1  00001.ppm   983   388  1024   432       40
2  00001.ppm   386   494   442   552       38
3  00001.ppm   973   335  1031   390       13
4  00002.ppm   892   476  1006   592       39


In [5]:
# Adding new empty columns to dataFrame to save numbers for YOLO format
ann['CategoryID'] = ''
ann['center x'] = ''
ann['center y'] = ''
ann['width'] = ''
ann['height'] = ''

In [6]:
# Getting category's ID according to the class's ID
# Writing numbers into appropriate column
ann.loc[ann['ClassID'].isin(p), 'CategoryID'] = 0
ann.loc[ann['ClassID'].isin(d), 'CategoryID'] = 1
ann.loc[ann['ClassID'].isin(m), 'CategoryID'] = 2
ann.loc[ann['ClassID'].isin(o), 'CategoryID'] = 3

In [7]:
# Calculating bounding box's center in x and y for all rows
# Saving results to appropriate columns
ann['center x'] = (ann['XMax'] + ann['XMin']) / 2
ann['center y'] = (ann['YMax'] + ann['YMin']) / 2

In [8]:
# Calculating bounding box's width and height for all rows
# Saving results to appropriate columns
ann['width'] = ann['XMax'] - ann['XMin']
ann['height'] = ann['YMax'] - ann['YMin']

In [9]:
r = ann.loc[:, ['ImageID',
                'CategoryID',
                'center x',
                'center y',
                'width',
                'height']].copy()

In [10]:
print(r.head())

     ImageID  CategoryID  center x  center y  width  height
0  00000.ppm           1     794.5     428.5     41      35
1  00001.ppm           2    1003.5     410.0     41      44
2  00001.ppm           2     414.0     523.0     56      58
3  00001.ppm           3    1002.0     362.5     58      55
4  00002.ppm           2     949.0     534.0    114     116


In [11]:
os.chdir(full_path_to_ts_dataset)
# print(os.getcwd())

In [12]:
# Using os.walk for going through all directories
# and files in them from the current directory
# Fullstop in os.walk('.') means the current directory
for current_dir, dirs, files in os.walk('.'):
    # Going through all files
    for f in files:
        # Checking if filename ends with '.ppm'
        if f.endswith('.ppm'):
            # Reading image and getting its real width and height
            image_ppm = cv2.imread(f)

            # Slicing from tuple only first two elements
            h, w = image_ppm.shape[:2]

            # Slicing only name of the file without extension
            image_name = f[:-4]

            # Getting Pandas dataFrame that has only needed rows
            # By using 'loc' method we locate needed rows
            # that satisfies condition 'classes['ImageID'] == f'
            # that is 'find from the 1st column element that is equal to f'
            # By using copy() we create separate dataFrame
            # not just a reference to the previous one
            # and, in this way, initial dataFrame will not be changed
            sub_r = r.loc[r['ImageID'] == f].copy()

            # Normalizing calculated bounding boxes' coordinates
            # according to the real image width and height
            sub_r['center x'] = sub_r['center x'] / w
            sub_r['center y'] = sub_r['center y'] / h
            sub_r['width'] = sub_r['width'] / w
            sub_r['height'] = sub_r['height'] / h

            # Getting resulted Pandas dataFrame that has only needed columns
            # By using 'loc' method we locate here all rows
            # but only specified columns
            # By using copy() we create separate dataFrame
            # not just a reference to the previous one
            # and, in this way, initial dataFrame will not be changed
            resulted_frame = sub_r.loc[:, ['CategoryID',
                                           'center x',
                                           'center y',
                                           'width',
                                           'height']].copy()

            # Checking if there is no any annotations for current image
            if resulted_frame.isnull().values.all():
                # Skipping this image
                continue

            # Preparing path where to save txt file
            path_to_save = os.path.join(full_path_to_ts_dataset + '../YOLO format')
            path_to_info = path_to_save + '/' + image_name + '.txt'

            # Saving resulted Pandas dataFrame into txt file
            resulted_frame.to_csv(path_to_info, header=False, index=False, sep=' ')

            # Preparing path where to save jpg image
            path_to_image = path_to_save + '/' + image_name + '.jpg'

            # Saving image in jpg format by OpenCV function
            # that uses extension to choose format to save with
            cv2.imwrite(path_to_image, image_ppm)

In [ ]:
cv2.__version__

In [ ]:
import ffmpeg